In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn import preprocessing, ensemble, model_selection

In [9]:
# Load data
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [71]:
# prepare training set
y_train = df_train.SalePrice.values.astype(float)

X_train = df_train.drop('SalePrice', axis=1)
X_train = X_train.fillna('noinfo')
X_train = pd.get_dummies(X_train)

# X_train = X_train.loc[:, X_train.columns[1:]].set_index(X_train.Id.values)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65,8450,Pave,noinfo,Reg,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,2,2008,WD,Normal
1,2,20,RL,80,9600,Pave,noinfo,Reg,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,5,2007,WD,Normal
2,3,60,RL,68,11250,Pave,noinfo,IR1,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,9,2008,WD,Normal
3,4,70,RL,60,9550,Pave,noinfo,IR1,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,2,2006,WD,Abnorml
4,5,60,RL,84,14260,Pave,noinfo,IR1,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,12,2008,WD,Normal
5,6,50,RL,85,14115,Pave,noinfo,IR1,Lvl,AllPub,...,0,0,noinfo,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75,10084,Pave,noinfo,Reg,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,8,2007,WD,Normal
7,8,60,RL,noinfo,10382,Pave,noinfo,IR1,Lvl,AllPub,...,0,0,noinfo,noinfo,Shed,350,11,2009,WD,Normal
8,9,50,RM,51,6120,Pave,noinfo,Reg,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,4,2008,WD,Abnorml
9,10,190,RL,50,7420,Pave,noinfo,Reg,Lvl,AllPub,...,0,0,noinfo,noinfo,noinfo,0,1,2008,WD,Normal


In [ ]:


# one hot encoding of categorical features

df_test = pd.get_dummies(df_test)

# creating matrices for sklearn
X_train = X_train.values
X_test = df_test.values

In [6]:
# prepare tuning params for grid search
nums_trees = [x for x in range(200, 2600, 100)]
loss_criteria = ['mse', 'mae']
feature_fractions = [0.1, 0.2, 0.3]
min_node_sizes = [x for x in range(5, 30, 5)]
tuning_params = {'n_estimators': nums_trees,
                 'criterion': loss_criteria,
                 'max_features': feature_fractions,
                 'min_samples_leaf': min_node_sizes}

In [7]:
# make a grid
clf = model_selection.GridSearchCV(ensemble.RandomForestRegressor(oob_score=True), tuning_params, cv=10)
clf.fit(X_train, y_train)
print("Best parameters set found on training set:")
print()
print(clf.best_params_)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').